In [180]:
import numpy as np
import pandas as pd;
import warnings
warnings.filterwarnings('ignore')

### Reading csv file

In [181]:
rawData = pd.read_csv("loan.csv")

### Understanding Data 
Viewing Top 5 Records  , shape of the data  & structure of the data with its datatype

In [182]:
rawData.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [183]:
rawData.shape

(39717, 111)

In [184]:
print(rawData.info(verbose=True, null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              39717 non-null  int64  
 1    member_id                       39717 non-null  int64  
 2    loan_amnt                       39717 non-null  int64  
 3    funded_amnt                     39717 non-null  int64  
 4    funded_amnt_inv                 39717 non-null  float64
 5    term                            39717 non-null  object 
 6    int_rate                        39717 non-null  object 
 7    installment                     39717 non-null  float64
 8    grade                           39717 non-null  object 
 9    sub_grade                       39717 non-null  object 
 10   emp_title                       37258 non-null  object 
 11   emp_length                      38642 non-null  object 
 12   home_ownership  

### Cleaning Data Set > Removing Irrelevant Columns

In [185]:
# if we look at above column list , we will find few of the fields that are irrelevant
# and should not be considered as part of our EDA process , Lets talk about each of them and understand 
# why they are not relevant

# id  - A unique LC assigned ID for the loan listing. it is not an usefull information and adds noise to the data 
# member_id - A unique LC assigned Id for the borrower member. it is again a random numeric number added and does not add value
# emp_title - The job title supplied by the Borrower when applying for the loan.Additionally this information got 
        # changed to employee name after 9/23/2013. Further it has text record with subjective information that 
        # does not add uniformity with each other.hence should be removed
# pymnt_plan - Indicates if a payment plan has been put in place for the loan . This has single value so analysis
        # can be performed as it has same value for all the rows
# url - URL for the LC page with listing data. It is just an information of the url and need not to be kept for analysis
# desc - Loan description provided by the borrower , it is not a useful information 
# zip_code - The first 3 numbers of the zip code provided by the borrower in the loan application. 
        # This is a masked information and so cannot be used 
# policy_code - publicly available policy_code=1,new products not publicly available policy_code=2. Dataset has single 
        # record since we are only analyzing public data here. Not Useful Information
# application_type - Indicates whether the loan is an individual application or a joint application with two co-borrowers.
        # Same Value present for all the rows  
#title - The loan title provided by the borrower , there are 19615 unique entries for this column and the
        #information is more a remarks and would not add value to the EDA 
#collections_12_mths_ex_med - Number of collections in 12 months excluding medical collections , it has unique values as 0 or NA
            # does not add value to eda
#tax_liens - Number of tax liens , it has unique values as 0 or NA, does not add value to eda
#chargeoff_within_12_mths - Number of charge-offs within 12 months,it has unique values as 0 or NA, does not add value to eda
#next_pymnt_d - Next scheduled payment date , it has 97% records missing and is not fit for EDA analysis on ground of that 
#mths_since_last_record - The number of months since the last public record. around 93% of data missing  

pickedData = rawData.drop(['id', 'member_id', 'emp_title','pymnt_plan','url','desc','zip_code',
                                'policy_code','application_type','title',"collections_12_mths_ex_med"
                          ,"tax_liens","chargeoff_within_12_mths","next_pymnt_d","mths_since_last_record"], axis=1)

On Looking Non null values of the above df.info() response , it is visible that there are columns in the dataset that have no values for all the rows, we need to first find these columns & drop them as they dont have any data. With this, we are now moving towards data cleaning part of our EDA 

### Cleaning Data Set > Finding Columns With Null Values

In [186]:
# compute percentage of missing values for each column
missing_values_percent = (pickedData.isnull().sum() / pickedData.shape[0] * 100).round(2).sort_values(ascending=False)
print(missing_values_percent)

open_il_12m              100.0
inq_last_12m             100.0
mort_acc                 100.0
mo_sin_rcnt_tl           100.0
mo_sin_rcnt_rev_tl_op    100.0
                         ...  
revol_bal                  0.0
pub_rec                    0.0
open_acc                   0.0
last_pymnt_amnt            0.0
loan_amnt                  0.0
Length: 96, dtype: float64


In [187]:
## Removing Columns that have 100% Null Values

# remove columns with zero missing values
missing_values_percent = missing_values_percent[missing_values_percent != 0]

# remove columns with 100% missing values
cols_to_drop = missing_values_percent[missing_values_percent == 100].index

dataAfterDroppingNullCol = pickedData.drop(cols_to_drop, axis=1)

print(dataAfterDroppingNullCol.shape)

(39717, 42)


On Dropping the columns , we have now have filtered data for 39717 rows and 48 columns. This we have done for columns ,now if we will go back for df.info() response , we will see that there are few of the columns that have less records, lets deep dive more in to these records

### Cleaning Data Set > Finding Columns With Missing Values

In [188]:
# Before Proceeding Further ,let us once again look at our dataset to pull missing values cols 
some_values_missing_cols = missing_values_percent[(missing_values_percent>0)&(missing_values_percent<100)]
print(some_values_missing_cols)

mths_since_last_delinq    64.66
emp_length                 2.71
pub_rec_bankruptcies       1.75
last_pymnt_d               0.18
revol_util                 0.13
last_credit_pull_d         0.01
dtype: float64


### Cleaning Data Set > Handling Missing Values

In [189]:
for key, value in some_values_missing_cols.iteritems():
    unique_values = dataAfterDroppingNullCol[key].unique()
    print('Unique values in {}: {}'.format(key, unique_values))

Unique values in mths_since_last_delinq: [ nan  35.  38.  61.   8.  20.  18.  68.  45.  48.  41.  40.  74.  25.
  53.  39.  10.  26.  56.  77.  28.  52.  24.  16.  60.  54.  23.   9.
  11.  13.  65.  19.  80.  22.  59.  79.  44.  64.  57.  14.  63.  49.
  15.  73.  70.  29.  51.   5.  75.  55.   2.  30.  47.  33.  69.   4.
  43.  21.  27.  46.  81.  78.  82.  31.  76.  62.  72.  42.  50.   3.
  12.  67.  36.  34.  58.  17.  71.  66.  32.   6.  37.   7.   1.  83.
  86. 115.  96. 103. 120. 106.  89. 107.  85.  97.  95.   0.]
Unique values in emp_length: ['10+ years' '< 1 year' '1 year' '3 years' '8 years' '9 years' '4 years'
 '5 years' '6 years' '2 years' '7 years' nan]
Unique values in pub_rec_bankruptcies: [ 0.  1.  2. nan]
Unique values in last_pymnt_d: ['Jan-15' 'Apr-13' 'Jun-14' 'May-16' 'Apr-12' 'Nov-12' 'Jun-13' 'Sep-13'
 'Jul-12' 'Oct-13' 'May-13' 'Feb-15' 'Aug-15' 'Oct-12' 'Sep-12' nan
 'Dec-12' 'Dec-14' 'Aug-13' 'Nov-13' 'Jan-14' 'Apr-14' 'Aug-14' 'Oct-14'
 'Aug-12' 'Jul-14' 'J

### Fixing Columns One By One 

### emp_length

In [190]:
# Lets pick the first column emp_length 
print(dataAfterDroppingNullCol['emp_length'].describe())
# the 'emp_length' column has 38,642 non-missing values, 11 unique values, 
# and '10+ years' is the most frequently occurring value in the column, occurring 8,879 times.
mode_value = dataAfterDroppingNullCol['emp_length'].mode()[0]
print(mode_value)
dataAfterDroppingNullCol['emp_length'].fillna(mode_value,inplace=True)

count         38642
unique           11
top       10+ years
freq           8879
Name: emp_length, dtype: object
10+ years


### mths_since_last_delinq

In [191]:
print(dataAfterDroppingNullCol['mths_since_last_delinq'].describe())
# This column defines the number of months since the borrower's last delinquency and empty values are around  36%
# Since this defines borrowers financial capability , we should not take mean , median but instead we should assume that missing
# values states that borrower hasnt done any such event , Lets impute the missing value with 0
dataAfterDroppingNullCol['mths_since_last_delinq'].fillna(0,inplace=True)

count    14035.000000
mean        35.900962
std         22.020060
min          0.000000
25%         18.000000
50%         34.000000
75%         52.000000
max        120.000000
Name: mths_since_last_delinq, dtype: float64


### pub_rec_bankruptcies

In [192]:
print(dataAfterDroppingNullCol['pub_rec_bankruptcies'].describe())
print(dataAfterDroppingNullCol['pub_rec_bankruptcies'].value_counts())
# This column defines Number of public record bankruptcies and empty values are around  1.75%
# Since this defines borrowers financial capability , we should not take mean , median but instead we should assume that missing
# values states that borrower hasnt done any such event , Lets impute the missing value with 0
dataAfterDroppingNullCol['pub_rec_bankruptcies'].fillna(0,inplace=True)

count    39020.000000
mean         0.043260
std          0.204324
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: pub_rec_bankruptcies, dtype: float64
0.0    37339
1.0     1674
2.0        7
Name: pub_rec_bankruptcies, dtype: int64


### last_pymnt_d               

In [196]:
print(dataAfterDroppingNullCol['last_pymnt_d'].describe())
print(dataAfterDroppingNullCol['last_pymnt_d'].value_counts())
# This column defines Last month payment was received and empty values are around  0.18%
# We can safely assume that entries that dont have dates for the payment recieved , have actually not recieved any payment
dataAfterDroppingNullCol['last_pymnt_d'].fillna("Missing Payment",inplace=True)

count      39717
unique       102
top       May-16
freq        1256
Name: last_pymnt_d, dtype: object
May-16    1256
Mar-13    1026
Dec-14     945
May-13     907
Feb-13     869
          ... 
Jun-08      10
Nov-08      10
Mar-08       5
Jan-08       4
Feb-08       1
Name: last_pymnt_d, Length: 102, dtype: int64


### revol_util

In [197]:
print(dataAfterDroppingNullCol['revol_util'].describe())
# This column defines Revolving line utilization rate, or the amount of credit the borrower is using relative to 
# all available revolving credit , so very high chances that the borrower has not done this event and hence value should 
# be replaced by 0% and would also like to mention that when we look at the mode value , the same is also 0%
# but had it not be same , we still should have gone with assumption that borrower has not performed the event 

mode_value = dataAfterDroppingNullCol['revol_util'].mode()[0]

print(mode_value)

dataAfterDroppingNullCol['revol_util'].fillna("0%",inplace=True)

count     39717
unique     1089
top          0%
freq       1027
Name: revol_util, dtype: object
0%


### last_credit_pull_d

In [195]:
print(dataAfterDroppingNullCol['last_credit_pull_d'].describe())
# This column defines The most recent month LC pulled credit for this loan  and the missing values are .01% , we can 
# assume that the borrower has not done this event and so the value amputed here should be "Missing Event"

mode_value = dataAfterDroppingNullCol['last_credit_pull_d'].mode()[0]
print(mode_value)

dataAfterDroppingNullCol['last_credit_pull_d'].fillna("Missing Event",inplace=True)

count      39715
unique       106
top       May-16
freq       10308
Name: last_credit_pull_d, dtype: object
May-16
